In [49]:
# !rm -rf '/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/code/Untitled Folder'

### Entities

In [2]:
import os
import pandas as pd

# homeFolder = "/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction"
# 
# Re-docred
data_path = "phi3/redocred_entities.csv"
# data_path = "openai/gpt4_redocred_dev_entity_detection_combined.csv"

# MC1
# data_path = "mistral7B/mistral7b_mc1_entities.csv"
# data_path = "openai/openai_mc1_entities_Apr13_processed.csv"
# data_path = "mistral7B/mistral7b_inst_mc1_entities_combined.csv"
# data_path = "universalNER/universalner_mc1_entities_processed.csv"
# data_path = "phi2/phi2_mc1_entities_processed.csv"

## SD Apr 25
# data_path = "mistral7B/it_sdapr25_entities.csv"
# data_path = "universalNER/sd_apr25_entities_processed.csv"
# data_path = "openai/sdapr25_entities.csv"

# data = pd.read_csv(data_path)

data = pd.read_csv(data_path, header=None)
data = data.rename(columns={0: 'index', 1: 'entities', 2: 'latency',
                            3: 'input_tokens',  4: 'output_tokens'})

# data = data.rename(columns={0: 'doc_key', 1: 'chunk_index', 2: 'entities', 3: 'latency',
#                             4: 'input_tokens',  5: 'output_tokens'})

output_file_name = "phi3/redocred_entities_processed.csv"

print(data.shape)
data.head()

(500, 5)


,index,entities,latency,input_tokens,output_tokens
0,0,"Wilfried ""Willi"" Schneider; Media, Transylvani...",3.031677,953,77
1,1,Alberta; Calgary; Ross Patterson Alger; August...,2.971230,1017,93
2,2,Jeff Healey; Mess of Blues; 2008; 42; Islingto...,2.093818,878,65
3,3,ASCAP; Ramey Idriss; Los Angeles Community Col...,2.992005,996,94
4,4,Escuela Latinoamericana de Medicina (ELAM); La...,3.451658,1003,109


#### Add openai tokens

In [63]:
import tiktoken

def num_tokens_from_string(string: str) -> int:
    encoding_name = "gpt-4"
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

print(num_tokens_from_string("Hello world, let's test tiktoken."))

9


In [64]:
data["output_tokens"] = ""
data["entities_v2"] = ""

for idx in data.index:
    data["output_tokens"][idx] = num_tokens_from_string(data["entities"][idx])
    
    if idx in [3, 13, 38, 450]:
        tmp = data["entities"][idx].replace("'ll", "ll")
        tmp = tmp.replace("State's", "States")
        tmp = tmp.replace(" ' Cup", " Cup")
        tmp = tmp.replace("O'Hara", " O Hara")
        
        tmp = eval(tmp)
        
    else:
        tmp = eval(data["entities"][idx])
        
    data["entities_v2"][idx] = "; ".join(tmp)
    
data.head()

/tmp/ipykernel_2861756/3635279979.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["output_tokens"][idx] = num_tokens_from_string(data["entities"][idx])
/tmp/ipykernel_2861756/3635279979.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["entities_v2"][idx] = "; ".join(tmp)


,original_index,entities,latency,output_tokens,entities_v2
0,0,"['Wilfried "" Willi "" Schneider', '13 March 196...",4.698511,130,"Wilfried "" Willi "" Schneider; 13 March 1963; M..."
1,1,"['Ross Patterson Alger', 'August 20, 1920', 'J...",6.246217,151,"Ross Patterson Alger; August 20, 1920; January..."
2,2,"['Mess of Blues', '2008', '42nd', 'Islington A...",2.543007,73,Mess of Blues; 2008; 42nd; Islington Academy; ...
3,3,"['Ramey Idriss', '11 September 1911', '5 Febru...",6.619722,154,Ramey Idriss; 11 September 1911; 5 February 19...
4,4,"['Escuela Latinoamericana de Medicina', 'Escue...",4.945866,149,Escuela Latinoamericana de Medicina; Escuela L...


In [65]:
data.to_csv("openai/gpt4_redocred_dev_entity_detection_combined_v2.csv", index=False)

In [54]:
# data["entities"][idx]

"['Frederick Jacob Titus Chiluba', 'April 30, 1943', 'June 18, 2011', 'Zambian', 'President of Zambia', '1991', '2002', 'Movement for Multi-party Democracy', 'Kenneth Kaunda', '1996', '2001', 'Vice President', 'Levy Mwanawasa', 'MMD', '2009']"

In [55]:
# eval(data["entities"][idx].replace("'ll", "ll"))

In [3]:
import re
import collections

if 'universalNER' in data_path:
    ent_col_name = "processed_entities"
    tok_col_name = "total_output_tokens"
    
elif data_path in ["phi2/phi2_mc1_entities_processed.csv"]:
    ent_col_name = "processed_entities"
    tok_col_name = "output_tokens"
    
elif "openai/gpt4_redocred_dev" in data_path:
    ent_col_name = "entities_v2"
    tok_col_name = "output_tokens"
    
else:
    ent_col_name = "entities"
    tok_col_name = "output_tokens"

entities_map = collections.defaultdict(list)
output_tokens = collections.defaultdict(int)
for idx in data.index:
    
    # doc_key = data["doc_key"][idx]
    doc_key = idx
    try:
        ent = data[ent_col_name][idx].strip().replace("\n", " ")
        ent = ent.split("; ")
    except:
        ent = ""
    
    entities_map[doc_key].extend(ent)
    output_tokens[doc_key] += data[tok_col_name][idx]
    # break

In [4]:
# skipping entities larger than length 30

ent_data = []
for k, v in entities_map.items():
    v = list(set(v))
    v_2 = []
    for item in v:
        item = item.replace(";", "").strip()
        
        if len(item) > 30 or not item:
            
            # print(item)
            # print("*"*10)
            continue
            
        elif len(item) == 4:
            
            tmp = re.sub("\d+", "", item)
            
            if not tmp:
                continue
            
        v_2.append(item)
    
    v_2 = "; ".join(v_2)
    v_2 = re.sub(";+", ";", v_2)
    ent_data.append([k, v_2, output_tokens[k]])


In [5]:
ent_data[32]

[32,
 'County; Norway; Rogfast; Terje Halleland; Rogaland; Janne Johnsen; County Governor; Rogaland County Council; Stavanger; Conservative Party; Harald Thune; Rogaland County Municipality; Rogaland County; King of Norway; Government of Norway; Progress Party',
 146]

In [6]:
df = pd.DataFrame(ent_data)
df.head()

,0,1,2
0,0,2002 Winter Olympics; Canadian skeleton team; ...,77
1,1,Ross Patterson Alger; alderman; Olympic colise...,93
2,2,Jeff Healey's Roadhouse; 42; Doc Pomus; Jeff H...,65
3,3,Los Angeles Community College; I'll Wait; Geor...,94
4,4,"19,550 students; Cuban healthcare system; 110 ...",109


In [7]:
output_file_name

'phi3/redocred_entities_processed.csv'

In [8]:
df = df.rename(columns={0: 'index', 1: 'processed_entities', 2: "output_tokens"})

df.to_csv(output_file_name, index=False)

### Triplets

In [12]:
import os
import pandas as pd

# Redocred
entiity_data_path = "/work/pi_dhruveshpate_umass_edu/project_19/ReDocREDPreprocessing/Re-DocRED/processed/Re-DocRED_Processed_Dev_v3.csv"

# data_path = "llama2/redocred_relations_combined.csv"
# data_path = "openai/openai_relations_gt_redocred_dev_all.csv"
# data_path = "phi2/phi2_relations_gt_redocred_dev_combined_v2.csv"
# data_path = "gemma/gemma2b_relations_gt_redocred_dev_combined.csv"
# data_path = "gemma/gemma2bit_relations_gt_redocred_dev.csv"
data_path = "phi3/redocred_relations.csv"

# MC1
# data_path = "openai/openai_mc1_triplets_combined.csv"
# entiity_data_path = "openai/openai_mc1_entities_Apr13_processed_docwise.csv"

# data_path = "mistral7B/mistral7b_mc1_triplets.csv"
# entiity_data_path = "mistral7B/mistral7b_mc1_entities_processed.csv"

# data_path = "mistral7B/mistral7b_inst_mc1_triplets_combined.csv"
# entiity_data_path = "mistral7B/mistral7b_inst_mc1_entities_processed.csv"

# SD 25Apr
# entiity_data_path = "../datasets/synthetic_data_Apr25/news_training_processed.csv"

# data_path = "mistral7B/it_sdapr25_relations.csv"
# data_path = "mistral7B/base_sdapr25_relations_combined.csv"
# data_path = "openai/sdapr25_relations_combined.csv"


# data = pd.read_csv(data_path)
# data = data.rename(columns={'entities': 'triplets'})

data = pd.read_csv(data_path, header=None)
data = data.rename(columns={0: 'index', 1: 'triplets', 2: 'latency',
                            3: 'input_tokens',  4: 'output_tokens', 'entities': 'triplets'})
# data = data.rename(columns={0: 'doc_key', 1: 'chunk_index', 2: 'triplets', 3: 'latency',
#                             4: 'input_tokens',  5: 'output_tokens', 'entities': 'triplets'})

output_file_name = "phi3/redocred_relations_processed.csv"

print(data.shape)
data.head()

(499, 5)


,index,triplets,latency,input_tokens,output_tokens
0,0,"Wilfried "" Willi "" Schneider | nationality | G...",7.734808,1603,368
1,1,Alberta | contains subnational entity | Calgar...,6.384342,1655,382
2,2,Jeff | album | Mess of Blues\nJeff Healey | re...,4.260506,1465,255
3,3,Ramey Idriss | nationality | American\nRamey I...,5.620533,1648,339
4,4,Escuela Latinoamericana de Medicina | located ...,5.836607,1617,350


### combine doc wise data

In [14]:
import re
import collections

trip_col_name = "triplets"
tok_col_name = "output_tokens"

triplet_map = collections.defaultdict(list)
output_tokens = collections.defaultdict(int)
for idx in data.index:
    
    # doc_key = data["doc_key"][idx]
    doc_key = idx
    try:
        ent = data[trip_col_name][idx].strip()
        ent = ent.split("\n")
        
        triplet_map[doc_key].extend(ent)
        output_tokens[doc_key] += data[tok_col_name][idx]
    except Exception as e:
        print(f"Error in {idx} - {e}")
        triplet_map[doc_key].extend([])
        output_tokens[doc_key] += 0

In [15]:
len(triplet_map.keys())

499

In [16]:
triplet_data = []
for k, v in triplet_map.items():
    v = list(set(v))
    v_2 = []
    for item in v:
        item = item.strip()
        
        if not item:
            continue
            
        v_2.append(item)
    
    # v_2 = "\n".join(v_2)
    triplet_data.append([k, v_2, output_tokens[k]])

len(triplet_data)

499

In [17]:
triplet_data[2]

[2,
 ["Jeff Healey's Roadhouse | location | Toronto",
  "Jeff Healey's Roadhouse | location | Canada",
  'Note: The relations output does not include the relations between entities that are not directly related to the given entity list. For example, "Jeff Healey | record label | Island Records" is not included because Island Records is not in the provided entity list.',
  'Jeff Healey | record label | Island Records',
  'Jeff Healey | age | 42',
  'Jeff Healey | recording location | Canada',
  'Jeff Healey | recording location | Studio 92',
  'Studio 92 | location | Canada',
  'Jeff Healey | live performance location | Islington Academy',
  'Jeff | album | Mess of Blues',
  'Islington Academy | location | London',
  '" Mess of Blues " | writer | Doc Pomus',
  'Jeff Healey | live performance location | Roadhouse in Toronto',
  'Jeff Healey | death | 2008',
  'Jeff Healey | songwriter | " Mess of Blues "',
  '" Mess of Blues " | writer | Mort Shuman',
  '" Mess of Blues " | original arti

### remove triplets with entities not in the list

In [18]:
## input entity data
entiity_data = pd.read_csv(entiity_data_path)
entiity_data = entiity_data.rename(columns={'Unnamed: 0': 'doc_key'})
print(entiity_data.shape)
entiity_data.head()

(500, 7)


,index,Title,Text,Triplets,Entities,triplet_tokens,entity_tokens
0,0,Willi Schneider (skeleton racer),"Wilfried "" Willi "" Schneider (born 13 March 19...",2002 Winter Olympics | start time | 2002\n2002...,German\nJeff Pain\nFIBT World Championships\n2...,611.0,85.0
1,1,Ross Alger,"Ross Patterson Alger (August 20, 1920 January...",Ross Patterson Alger | place of birth | Prelat...,Rod Sykes\nOlympic\nRoss Patterson Alger\nRoya...,550.0,72.0
2,2,Mess of Blues (Jeff Healey album),Mess of Blues is an album by Jeff Healey. It w...,Mess of Blues | publication date | 2008\nMess ...,Toronto\nDoc Pomus\nCanada\nStudio 92\nIslingt...,189.0,39.0
3,3,Ramey Idriss,Ramey Idriss (11 September 1911 5 February 19...,Wet Blanket Policy | publication date | 1948\n...,The Old Chaperone\nI 'll Wait\n11 September 19...,564.0,94.0
4,4,ELAM (Latin American School of Medicine) Cuba,"Escuela Latinoamericana de Medicina (ELAM), fo...",Latin American School of Medicine | country | ...,Guri\nCuba\nLatin America\nEscuela Latinoameri...,417.0,66.0


In [68]:
entiity_data["processed_entities"][2]

'Michael Zennie; sightseer; Yellowstone; Grand Prismatic Spring; bacteria; algae; minerals; Park officials; National Park Service; drones; Yellowstone National Park; Wyoming; Al Nash; Jeffrey Olson; United States; Zion; Yosemite; park newspaper; rangers; helicopter'

In [19]:
entiity_data_map = {}
for idx in entiity_data.index:
    # print(entiity_data["doc_key"][idx])
    try:
        # entiity_data_map[entiity_data["doc_key"][idx]] = entiity_data["processed_entities"][idx].lower().split("; ")
        entiity_data_map[idx] = entiity_data["Entities"][idx].lower().split("\n")
    except:
        # entiity_data_map[entiity_data["doc_key"][idx]] = ""
        entiity_data_map[idx] = ""

In [20]:
entiity_data_map[2]

['toronto',
 'doc pomus',
 'canada',
 'studio 92',
 'islington academy',
 'roadhouse',
 'roadhouse in toronto',
 'jeff',
 'mess of blues',
 'london',
 'elvis presley',
 '2008']

In [21]:
# check triplet data
triplet_data_v2 = []
ct_edges_w_ent_not_present = 0
all_ent_not_present = set()

for item in triplet_data:
    doc_key = item[0]
    all_trips = []
    if doc_key == 847:
        break
    entities = entiity_data_map[doc_key]
    not_present = set()
    
    for tmp_trip_1 in item[1]:
        
        tmp_trip = tmp_trip_1.lower().split(" | ")
        if len(tmp_trip) != 3:
            continue
            
        flag = True
        
        if tmp_trip[0] not in entities or tmp_trip[2] not in entities:
            
            count = 0
            for ent_tmp in entities:
                
                if ent_tmp in tmp_trip[0] or ent_tmp in tmp_trip[2]:
                    count += 1
                    
            if count < 2:
                flag = False
            
        if not flag:
            
            if tmp_trip[0] not in entities:
                not_present.add(tmp_trip[0])
                
            if tmp_trip[2] not in entities:
                not_present.add(tmp_trip[2])
                
            ct_edges_w_ent_not_present += 1
                
            # print(tmp_trip)
            # print(entities)
            # print("*"*20)
            
        else:
            all_trips.append(tmp_trip_1)
            
    all_ent_not_present.update(not_present)
    triplet_data_v2.append([item[0], "\n".join(all_trips), item[2], len(not_present)])
    
ct_edges_w_ent_not_present, len(all_ent_not_present)

(2769, 1991)

In [22]:
doc_key

498

In [23]:
entiity_data_map

{0: ['german',
  'jeff pain',
  'fibt world championships',
  '2002',
  '2010 winter olympics',
  'wilfried " willi " schneider',
  'duff gibson',
  'canadian',
  'transylvania',
  '2006 winter olympics',
  'melissa hollingsworth',
  'turin',
  '2002 winter olympics',
  'skeleton world cup',
  'british columbia',
  'canada',
  'mediaș',
  'vancouver',
  'salt lake city',
  '13 march 1963',
  'jon montgomery'],
 1: ['rod sykes',
  'olympic',
  'ross patterson alger',
  'royal canadian air force',
  'prelate',
  '1992',
  'world war ii',
  'ctrain',
  'january 16 , 1992',
  'canadian',
  'august 20 , 1920',
  'harry alger',
  'xv olympic winter games',
  'university of toronto',
  'calgary city council',
  'calgary',
  'alberta',
  'university of alberta',
  'saskatchewan'],
 2: ['toronto',
  'doc pomus',
  'canada',
  'studio 92',
  'islington academy',
  'roadhouse',
  'roadhouse in toronto',
  'jeff',
  'mess of blues',
  'london',
  'elvis presley',
  '2008'],
 3: ['the old chaperone

In [24]:
triplet_data_v2[2]

[2,
 'Jeff Healey\'s Roadhouse | location | Toronto\nJeff Healey\'s Roadhouse | location | Canada\nJeff Healey | recording location | Canada\nJeff Healey | recording location | Studio 92\nStudio 92 | location | Canada\nJeff Healey | live performance location | Islington Academy\nJeff | album | Mess of Blues\nIslington Academy | location | London\n" Mess of Blues " | writer | Doc Pomus\nJeff Healey | live performance location | Roadhouse in Toronto\nJeff Healey | death | 2008\nJeff Healey | songwriter | " Mess of Blues "\n" Mess of Blues " | original artist | Elvis Presley',
 255,
 7]

In [26]:
df = pd.DataFrame(triplet_data_v2)
df = df.rename(columns={0: 'index', 1: 'processed_relations', 2: "output_tokens", 3: "entities_not_from_list"})
print(df.shape)
df.head()

(499, 4)


,index,processed_relations,output_tokens,entities_not_from_list
0,0,"Wilfried "" Willi "" Schneider | competed in | 2...",368,2
1,1,Ross Patterson Alger | mayor of | Calgary\nNot...,382,1
2,2,Jeff Healey's Roadhouse | location | Toronto\n...,255,7
3,3,I 'll Wait | composer | Ramey Idriss\nMy Three...,339,0
4,4,Cuba | location of | Escuela Latinoamericana d...,350,1


In [27]:
output_file_name

'phi3/redocred_relations_processed.csv'

In [28]:
df.to_csv(output_file_name, index=False)